## Examples of Python Itertools



Source: https://realpython.com/python-itertools/

#### The zip and map builtins

zip - combines elements from two iterables into a tuple of each element by index. So the first elements of each iterable are paired together, as are the second, and so on.
map - applies a given function to each element in an iterable

In [1]:
print("Example of zip:")
print(list(zip(["a", "b", "c"], [1, 2, 3])))

print("\nExample of map:")
print(list(map(len, ([1,2,3], [2], range(10)))))

print("\nExample of combining map and zip:")
print(list(map(sum, zip([1, 2, 3], [4, 5, 6]))))
[5, 7, 9]

Example of zip:
[('a', 1), ('b', 2), ('c', 3)]

Example of map:
[3, 1, 10]

Example of combining map and zip:
[5, 7, 9]


[5, 7, 9]

### Performance improvements using itertools

In [2]:
def naive_grouper(inputs, n):
    num_groups = len(inputs) // n
    return [tuple(inputs[i*n:(i+1)*n]) for i in range(num_groups)]

nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
naive_grouper(nums, 2)

[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]

In [19]:
%%timeit -r 3

for _ in naive_grouper(range(10_000_000), 10):
    pass

1.72 s ± 55.1 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [34]:
def better_grouper(inputs, n):
    # Create n copies of references to the inputs
    iters = [iter(inputs)] * n
    
    # When zip these references together. As zip pulls an element from an iterable, it does so for all of the references.
    # So when the first iterable is taken from zip's second input, this is now the second element of the original list. 
    return zip(*iters)

In [35]:
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list(better_grouper(nums, 2))

[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]

In [39]:
%%timeit -r 3

def better_grouper(inputs, n):
    iters = [iter(inputs)] * n
    return zip(*iters)

# This version is approximately 4 times faster
for _ in better_grouper(range(10_000_000), 10):
    pass

444 ms ± 9.51 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


`better_grouper` above fails to return all elements when n is not a factor of the length of the input list.
The reason for this is that zip stops when it exhausts the shortest iterable passed to it.
`itertools` zip_longest overcomes this problem.

In [40]:
nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list(better_grouper(nums, 4))

[(1, 2, 3, 4), (5, 6, 7, 8)]

In [41]:
import itertools as it


def grouper(inputs, n, fillvalue=None):
    iters = [iter(inputs)] * n
    return it.zip_longest(*iters, fillvalue=fillvalue)

nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(list(grouper(nums, 4)))

[(1, 2, 3, 4), (5, 6, 7, 8), (9, 10, None, None)]


### Example - Ways to make change from combinations of notes

Given, USD bills as listed in bills below, how many ways can you make change for a $100 bill?


For this we can use the `combinations` function from itertools. 

In [43]:
bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]
# list(it.combinations(bills, 3))

makes_100 = []
for n in range(1, len(bills) + 1):
    for combination in it.combinations(bills, n):
        if sum(combination) == 100:
            makes_100.append(combination)

In [45]:
set(makes_100)

{(20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 10)}

Here’s a variation on the same problem:

``` How many ways are there to make change for a $100 bill using any number of $50, $20, $10, $5, and $1 dollar bills? ```

For this we can use `it.combinations_with_replacement`

In [60]:
list(it.combinations_with_replacement(bills, 2))

[(10, 10), (10, 5), (10, 1), (5, 5), (5, 1), (1, 1)]

In [64]:
bills = [10, 5, 1]
makes_100 = []

value = 50
for n in range(1, 20):
    for combination in it.combinations_with_replacement(bills, n):
        sum(combination)
        if sum(combination) == value:
            makes_100.append(combination)

In [65]:
makes_100

[(10, 10, 10, 10, 10),
 (10, 10, 10, 10, 5, 5),
 (10, 10, 10, 5, 5, 5, 5),
 (10, 10, 5, 5, 5, 5, 5, 5),
 (10, 5, 5, 5, 5, 5, 5, 5, 5),
 (10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (5, 5, 5, 5, 5, 5, 5, 5, 5, 5),
 (10, 10, 10, 5, 5, 5, 1, 1, 1, 1, 1),
 (10, 10, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1),
 (10, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1),
 (10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1),
 (10, 10, 10, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (10, 10, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (10, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (10, 10, 10, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)]

`permuations` example

In [66]:
list(it.permutations(['a', 'b', 'c']))

[('a', 'b', 'c'),
 ('a', 'c', 'b'),
 ('b', 'a', 'c'),
 ('b', 'c', 'a'),
 ('c', 'a', 'b'),
 ('c', 'b', 'a')]

### `itertools.count`

In [69]:
evens = it.count(step=2)
list(next(evens) for _ in range(5))

[0, 2, 4, 6, 8]

In [70]:
odds = it.count(start=1, step=2)
list(next(odds) for _ in range(5))

[1, 3, 5, 7, 9]

In [71]:
count_with_floats = it.count(start=0.5, step=0.75)
list(next(count_with_floats) for _ in range(5))

[0.5, 1.25, 2.0, 2.75, 3.5]

### Recurrence Relations


Example Fibonacci numbers

In [72]:
# Example with generator:

def fibs():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [74]:
list(next(fibs()) for _ in range(5))

[0, 0, 0, 0, 0]

In [82]:
next(fibs())

0